# Projeto 2 - Classificador Automático de Sentimento

## Integrantes: Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___
## Introdução

Vivemos na era da informação, tudo pode ser usado para o bem ou para o mal, desde o que o indíviduo tá sentido, seja relacionado a uma empresa, produto ou até por alguma questão qualquer, o lugar por onde as pessoas passam, o que curtem em uma rede social, tudo pode ser usado para entender o comortamento social de um nicho específico da sociedade ou até traçar uma média geral para a nação. Com o avanço do conhecimento e do desenvolvimento do poderio da tecnologia, o ser humano percebeu que não precisa depender apenas de si, até por que ele mesmo é muito falho, mas podemos nos utilizar de máquinas não só para registrar, mas para pensar. Assim nasceu o conhecido 'Machine Learning', área da computação que visa programar um computador para aprender com o humano e por meio de probabilidades realizar o mesmo trabalho.

Neste trabalho, utilizamos de uma técnica muito simples de Machine Learning, chamada "Naive Bayes" (traduzido literalmente para Bayes ingênuo). O nome faz uma homenagem a Thomas Bayes (1701 – 1761), matemático inglês que estudou como calcular a distribuição para o parâmetro de probabilidade de uma distribuição binomial e acabou por chegar na famosa equação de probabilidades, a equação de Bayes:

### $$P(A | B) = \frac{P(B | A)·P(A)}{P(B)}$$

Contudo, nosso objetivo é analisar qual a reputação da empresa 'Riot Games', com base no seu público no twitter, para isso recolhemos um dataframe de 300 tweets de treinamento para que o algoritmo desenvolvido possa tomar como base e 700 tweets de teste para verificarmos o quão preciso ele é. A importância de um projeto como esse para uma empresa seria a informação estatística de como ela é vista pelo público em determinada rede social para se embasar em decisões futuras de investimentos, marketing, etc.

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
### Primeiros passos...

O primeiro passo para criar o classificador Naive-Bayes é importar nosso dataframe, para isso devemos escolher qual rede social iremos trabalhar e no caso será o Twitter. A partir de uma api, conseguimos acesso para extrair um dataframe em excel com diversos 'Tweets' (postagens desta rede em específico). Também devemos importar as bibliotecas que utilizaremos como pandas e matplotlib. O tema escolhido para o projeto é Hearthstone, um jogo de cartas online, um dos grandes eSports da atualidade.

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:
df=pd.read_excel('tweets_rappi.xlsx','Treinamento',sep=',')
dfTeste=pd.read_excel('tweets_rappi.xlsx', 'Teste',sep=',')

Em seguida, devemos "limpar" o dataframe, ou seja, remover parênteses, colchetes, etc. Estes caracteres interferem na análise das palavras e devem necessariamente ser removidos dos nossos tweets. Também iremos transformar os tweets em listas de strings, a separação de cada string se dá pelo espaço entre as palavras após a limpeza das mesmas.

In [3]:
lista1=['  ','   ',':','(',')','`','[',']','.','/','" ',"'"]
lista2=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
for i in range(len(lista1)):
    df['Treinamento']=df.Treinamento.str.replace(lista1[i], lista2[i])
    dfTeste['Teste']=dfTeste.Teste.str.replace(lista1[i], lista2[i])
df['Treinamento']=df.Treinamento.str.lower()
dfTeste['Teste']=dfTeste.Teste.str.lower()

In [4]:
df['Treinamento']=df['Treinamento'].str.split()
dfTeste['Teste']=dfTeste['Teste'].str.split()
df['Treinamento'].head()

0    [ganhe, r$100, em, frete, no, seus, primeiros,...
1    [@viniciusprimo_, @ifood, @rappibrasil, btw, a...
2    [rappi, fez, o, meu, dia, me, dando, 30, reais...
3    [todo, mundo, deve, achar, que, morar, sozinha...
4    [um, timing, perfeito, @mizanzika, @luide, tiv...
Name: Treinamento, dtype: object

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

___
### O que precisamos?
Agora chegou a hora de estruturar nossa função que calculará o nosso Bayes. Para isso precisamos das seguintes probabilidades:

<br>Probabilidade da palavra ser Irrelevante dentro do espaço amostral de todas as palavras.</br>
<br>Probabilidade da palavra ser Ruim dentro do espaço amostral de todas as palavras.</br>
<br>Probabilidade da palavra ser Neutra dentro do espaço amostral de todas as palavras.</br>
<br>Probabilidade da palavra ser Boa dentro do espaço amostral de todas as palavras.</br>

Além disso, precisaremos das probabilidades condicionais individuais das palavras dado sua categoria (bom, ruim, neutro ou irrelevante). Para esse cálculo precisamos ver quantas vezes cada palavra aparece na categoria em específico dividido pelo número de palavras totais classificadas nessa categoria. Porém nem tudo são flores, existe a possibilidade de no teste de tweets novos, aparecerem palavras nunca antes vistas pelo nosso algoritmo, mas existe uma solução graças a La Place! A técnica que utilizaremos se chama "Laplace smoothing" e ela se trata de, na probabilidade condicional da palavra dado a categoria, adicionar 1 ao numerador e adicionar o número de palavras totais no treinamento sem repetições ao denominador. Sempre que aparecer uma palavra nova, devemos adicionar isso e assim ele não alterará muito a classificação do tweet, dando uma probabilidade de acerto maior ao classificador.

Então vamos ao código...
___

In [5]:
dfLen=len(df)

dfBom=df[df['Classificação']==3]
dfBomLen=len(dfBom)

dfNeutro=df[df['Classificação']==2]
dfNeutroLen=len(dfNeutro)

dfRuim=df[df['Classificação']==1]
dfRuimLen=len(dfRuim)

dfIrre=df[df['Classificação']==0]
dfIrreLen=len(dfIrre)

In [6]:
# Contagem de palavras dado sua categoria
#Bom
dfBomProb=dfBom["Treinamento"].sum()
dfBomProb=pd.Series(dfBomProb)
dfBomProb=dfBomProb.value_counts()

#Ruim
dfRuimProb=dfRuim["Treinamento"].sum()
dfRuimProb=pd.Series(dfRuimProb)
dfRuimProb=dfRuimProb.value_counts()

#Irrelevante
dfIrreProb=dfIrre["Treinamento"].sum()
dfIrreProb=pd.Series(dfIrreProb)
dfIrreProb=dfIrreProb.value_counts()

#Neutro
dfNeutroProb=dfNeutro["Treinamento"].sum()
dfNeutroProb=pd.Series(dfNeutroProb)
dfNeutroProb=dfNeutroProb.value_counts()

#Total
dfTotal=df.Treinamento.sum()
dfTotal=pd.Series(dfTotal)
dfTotal=dfTotal.value_counts()

In [7]:
#Número total de palavras no dataframe
dfPalavras=df.Treinamento.sum()
dfPalavras=pd.Series(dfPalavras)
dfPalavras=len(dfPalavras)

#Check de palavras
check=df.Treinamento.sum()
check=pd.Series(check)

In [8]:
#Números totais de palavras em cada categoria, mas agora em variáveis
#Bom
BoasPalavras=dfBom["Treinamento"].sum()
BoasPalavras=pd.Series(BoasPalavras)
BoasPalavras=len(BoasPalavras)

#Ruim
RuimPalavras=dfRuim["Treinamento"].sum()
RuimPalavras=pd.Series(RuimPalavras)
RuimPalavras=len(RuimPalavras)

#Irrelevante
IrrePalavras=dfIrre["Treinamento"].sum()
IrrePalavras=pd.Series(IrrePalavras)
IrrePalavras=len(IrrePalavras)

#Neutro
NeutroPalavras=dfNeutro["Treinamento"].sum()
NeutroPalavras=pd.Series(NeutroPalavras)
NeutroPalavras=len(NeutroPalavras)


In [9]:
# Probabilidades base
ProbBom=BoasPalavras/(dfPalavras)
ProbRuim=RuimPalavras/(dfPalavras)
ProbIrre=IrrePalavras/(dfPalavras)
ProbNeutro=NeutroPalavras/(dfPalavras)

print('A probablidade de ser Irrelevante é: {0:.3f} %'.format(ProbIrre*100))
print('A probablidade de ser Ruim é: {0:.3f} %'.format(ProbRuim*100))
print('A probablidade de ser Neutro é: {0:.3f} %'.format(ProbNeutro*100))
print('A probablidade de ser Bom  é: {0:.3f} %'.format(ProbBom*100))

A probablidade de ser Irrelevante é: 13.863 %
A probablidade de ser Ruim é: 21.935 %
A probablidade de ser Neutro é: 39.255 %
A probablidade de ser Bom  é: 24.555 %


In [10]:
#Técnica laplace 3 (74.66% de acerto no treinamento)
g=[]
for i in check:
    if i not in g:
        g.append(i)
lala=len(g)
dfBomProb2=(dfBomProb+1)/(BoasPalavras+lala)
dfRuimProb2=(dfRuimProb+1)/(RuimPalavras+lala)
dfIrreProb2=(dfIrreProb+1)/(IrrePalavras+lala)
dfNeutroProb2=(dfNeutroProb+1)/(NeutroPalavras+lala)

In [11]:
resultado=[]
def bayes(df,dfPalavras,lista):
    laplace=(1/(lala))# obs da folha
    for i in df: # for i in df.index:
        c_bom=1
        c_ruim=1
        c_irre=1
        c_neutro=1
        for u in i:
            #p=dfTotal[u]/len(dfTotal) #(Probabilidade da palavra no espaço amostral de todas as palavras)
            if u not in dfBomProb2:
                p_bom=laplace
            else:
                p_bom=dfBomProb2[u]
            if u not in dfRuimProb2:
                p_ruim=laplace
            else:
                p_ruim=dfRuimProb2[u]
            if u not in dfIrreProb2:
                p_irre=laplace
            else:
                p_irre=dfIrreProb2[u]
            if u not in dfNeutroProb2:
                p_neutro=laplace
            else:
                p_neutro=dfNeutroProb2[u]
                
            c_bom*=(p_bom)
            c_ruim*=(p_ruim)
            c_irre*=(p_irre)
            c_neutro*=(p_neutro)
        c_bom=c_bom*(ProbBom)
        #print("c_bom:",c_bom)
        c_ruim=c_ruim*(ProbRuim)
        #print("c_ruim:",c_ruim)
        c_irre=c_irre*(ProbIrre)
        #print("c_irre:",c_irre)
        c_neutro=c_neutro*(ProbNeutro)

        if c_bom>c_ruim and c_bom>c_irre and c_bom>c_neutro:
            #Classifica como Bom (3)
            lista.append(3)

        elif c_ruim>c_bom and c_ruim>c_irre and c_ruim>c_neutro:
            #Classifica como Ruim (1)
            lista.append(1)
        elif c_irre>c_bom and c_irre>c_ruim and c_irre>c_neutro:
            #Classifica como irrelevante (0)
            lista.append(0)
        else:
            #Classifica como neutro (2) 
            lista.append(2)
    return lista

Agora que nossa função Bayes está pronta, podemos testar ela com o nosso próprio treinamento. No caso

In [12]:
a=bayes(df["Treinamento"],dfPalavras,resultado)

In [13]:
print(len(resultado))
print(a)

300
[2, 1, 3, 3, 0, 1, 3, 3, 3, 1, 2, 3, 2, 1, 2, 2, 1, 1, 1, 1, 1, 3, 1, 2, 2, 2, 3, 2, 3, 1, 3, 2, 2, 2, 0, 3, 2, 2, 3, 1, 1, 2, 3, 2, 2, 2, 1, 1, 3, 2, 1, 1, 2, 2, 1, 3, 3, 3, 1, 3, 3, 1, 3, 2, 3, 2, 3, 1, 1, 2, 1, 2, 1, 2, 0, 2, 2, 0, 3, 2, 3, 2, 2, 2, 1, 1, 3, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3, 3, 2, 1, 1, 1, 1, 2, 1, 3, 1, 0, 2, 3, 1, 3, 3, 1, 3, 1, 3, 2, 1, 3, 2, 3, 2, 2, 1, 2, 2, 1, 1, 3, 2, 2, 3, 2, 3, 1, 1, 1, 2, 0, 3, 3, 1, 2, 2, 2, 0, 3, 3, 2, 2, 2, 3, 3, 1, 1, 1, 1, 2, 0, 2, 1, 2, 1, 2, 1, 3, 3, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 1, 1, 1, 3, 1, 3, 3, 3, 1, 2, 2, 1, 2, 3, 1, 1, 0, 2, 3, 2, 2, 2, 3, 0, 2, 3, 3, 3, 3, 3, 1, 1, 1, 3, 2, 3, 2, 2, 1, 3, 3, 1, 3, 3, 3, 2, 3, 0, 3, 1, 0, 2, 3, 3, 2, 1, 3, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 0, 0, 0, 2, 2, 1, 2, 3, 1, 1, 2, 1, 2, 1, 3, 2, 3, 2, 2, 3, 3, 1, 2, 1, 1, 0, 2, 2, 1, 2, 2, 0, 2, 1, 1, 3, 2, 2, 2, 3, 0, 2, 3, 2, 2]


In [14]:
df["Resultado"]=a

In [15]:
df.head()

,Treinamento,Classificação,Legenda,Resultado
0,"[ganhe, r$100, em, frete, no, seus, primeiros,...",2.0,0 - IRRELEVANTE,2
1,"[@viniciusprimo_, @ifood, @rappibrasil, btw, a...",2.0,1 - RUIM,1
2,"[rappi, fez, o, meu, dia, me, dando, 30, reais...",3.0,2 - NEUTRO,3
3,"[todo, mundo, deve, achar, que, morar, sozinha...",2.0,3 - BOM,3
4,"[um, timing, perfeito, @mizanzika, @luide, tiv...",2.0,NaN,0


In [16]:
xx=pd.crosstab(df['Classificação'], df['Resultado'])
xx

Resultado,0,1,2,3
Classificação,,,,
0.0,17,6,19,8
1.0,1,60,5,3
2.0,1,10,77,13
3.0,0,4,16,59


In [17]:
acerto=(xx[0][0]+xx[1][1]+xx[2][2]+xx[3][3])/(sum(xx[0])+sum(xx[1])+sum(xx[2])+sum(xx[3]))
print("Taxa de Acerto: {:.3f} %".format(acerto*100))

Taxa de Acerto: 71.237 %


In [18]:
def verifica(ct, categorias): #ct = Crosstab ; categoria = 'string' (ruim, bom, etc)
    dicio={}
    for i in categorias:
        if i=='Irrelevante':
            verdadeiro=(ct[0][0])/sum(ct[0])
            falso=(sum(ct[0])-ct[0][0])/sum(ct[0])
        elif i=='Ruim':
            verdadeiro=(ct[1][0])/sum(ct[1])
            falso=(sum(ct[1])-ct[1][0])/sum(ct[1])
        elif i=='Neutro':
            verdadeiro=(ct[2][0])/sum(ct[2])
            falso=(sum(ct[2])-ct[2][0])/sum(ct[2])
        elif i=='Bom':
            verdadeiro=(ct[3][0])/sum(ct[3])
            falso=(sum(ct[3])-ct[3][0])/sum(ct[3])
        print('Taxa de {0} Verdadeiro: {1:.3f} %'.format(i,verdadeiro*100))
        print('Taxa de {0} Falso: {1:.3f} %'.format(i,falso*100))
        print('-'*25)
        dicio[i]=[verdadeiro,falso]
    return(dicio)
cat=['Irrelevante','Ruim','Neutro','Bom']
print('Testando no treinamento:\n')
ver=verifica(xx,cat)
print(ver)

Testando no treinamento:

Taxa de Irrelevante Verdadeiro: 89.474 %
Taxa de Irrelevante Falso: 10.526 %
-------------------------
Taxa de Ruim Verdadeiro: 7.500 %
Taxa de Ruim Falso: 92.500 %
-------------------------
Taxa de Neutro Verdadeiro: 16.239 %
Taxa de Neutro Falso: 83.761 %
-------------------------
Taxa de Bom Verdadeiro: 9.639 %
Taxa de Bom Falso: 90.361 %
-------------------------
{'Irrelevante': [0.89473684210526316, 0.10526315789473684], 'Ruim': [0.074999999999999997, 0.92500000000000004], 'Neutro': [0.1623931623931624, 0.83760683760683763], 'Bom': [0.096385542168674704, 0.90361445783132532]}


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [ ]:
r=[]
b=bayes(dfTeste['Teste'], dfPalavras,r)
dfTeste['Resultado']=b

In [ ]:
ct_teste=pd.crosstab(dfTeste['Classificação'],dfTeste['Resultado'])
ct_teste

In [ ]:
ver_teste=verifica(ct_teste,cat)
ver_teste

In [ ]:
acerto2=(ct_teste[0][0]+ct_teste[1][1]+ct_teste[2][2])/(sum(ct_teste[0])+sum(ct_teste[1])+sum(ct_teste[2]))
print("Taxa de Acerto: {:.3f} %".format(acerto*100))

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


___
### Como podemos melhorar?

Para melhor o nosso machine learning, uma coisa importante seria aumentar nossa base de treinamento e classifica-la com a maior proximidade com a realidade possível. Outro ponto seria aumentar uma categoria, para que assim tivessemos mais possibilidades de 'Sentimentos' atrelados a cada postagem, no caso seria Irrelevante, Ruim, Neutro, Bom e Muito Bom, o que poderia permitir uma análise mais profunda dos dados. Para melhorar a eficiência do código, poderiamos trabalhar com uma classe geral que recebe um dataframe classificado e retorna uma análise dos dados, além da classificação do Naive-Bayes. Poderíamos usar dicionários ao invés de listas pois otimizariam o tempo de execução do código. Outra coisa que podíamos implementar seria filtrar emojis e separa-los como strings para que assim eles fossem avaliados individualmente como se fossem palavras e trariam mais certeza ao código.

Porém essas são algumas hipóteses que nosso grupo teve para otimizar este projeto em futuras iterações.